# **ETL - POBLACION**

In [2]:
# [Configuración] Importación de librerías
import pandas as pd
import numpy as np
import os
import unicodedata
import re

In [3]:
# [Configuración] Definición de rutas - cargar datos de población
ruta_dep_1 = "../DATA/Poblacion/poblacion_departamental_2005-2017.csv"
ruta_dep_2 = "../DATA/Poblacion/poblacion_departamental_2018-2050.csv"
ruta_mun_1 = "../DATA/Poblacion/poblacion_municipal_2005_2017.csv"
ruta_mun_2 = "../DATA/Poblacion/poblacion_municipal_2018_2042.csv"

rutas = [ruta_dep_1, ruta_dep_2, ruta_mun_1, ruta_mun_2]
nombres = ['dep_1', 'dep_2', 'mun_1', 'mun_2']
dataframes = {}
for nombre, ruta in zip(nombres, rutas):
    df = pd.read_csv(ruta, encoding='latin_1', low_memory=False)
    dataframes[nombre] = df



data_dpto = pd.concat([dataframes['dep_1'], dataframes['dep_2']], ignore_index=True)
data_mun = pd.concat([dataframes['mun_1'], dataframes['mun_2']], ignore_index=True)

In [4]:
# Renombrar columnas
nombres = {
    0: 'ID_DPTO',
    1: 'DEPARTAMENTO',
    2: 'AÑO',
    3: 'ÁREA GEOGRÁFICA',
    4: 'POBLACIÓN',
    5: 'TOTAL'
}
data_dpto.columns = [nombres[i] for i in range(len(data_dpto.columns))]

nombres_por_indice = {
    0: 'ID_DPTO',
    1: 'DEPARTAMENTO',
    2: 'MUNICIPIO',
    3: 'ID_MUN',
    4: 'AÑO',
    5: 'ÁREA GEOGRÁFICA',
    6: 'POBLACIÓN'
}
data_mun.columns = [nombres_por_indice[i] for i in range(len(data_mun.columns))]

## Población nacional por departamento

In [5]:
data_dpto = data_dpto[(data_dpto["AÑO"] >= 2013) & (data_dpto["AÑO"] <= 2023)]

data_dpto = data_dpto[data_dpto["ÁREA GEOGRÁFICA"] == "Total"]

data_dpto['DEPARTAMENTO'] = data_dpto['DEPARTAMENTO'].str.strip().str.upper()
data_dpto['DEPARTAMENTO'] = data_dpto['DEPARTAMENTO'].replace(
    'ARCHIPIÉLAGO DE SAN ANDRÉS',
    'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA'
).replace(
    'QUINDIO', 'QUINDÍO'
)

data_dpto['POBLACIÓN'] = data_dpto['POBLACIÓN'].fillna(data_dpto['TOTAL'])
data_dpto = data_dpto.drop(columns=['ÁREA GEOGRÁFICA', 'TOTAL']).reset_index(drop=True)

# Ordenar por AÑO y luego por DEPARTAMENTO
data_dpto = data_dpto.sort_values(by=['AÑO', 'DEPARTAMENTO'], ascending=[True, True])
data_dpto['POBLACIÓN'] = (data_dpto['POBLACIÓN'].astype(str).str.replace(',', '', regex=False).astype(int))


In [6]:
data_dpto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 140 to 362
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID_DPTO       363 non-null    object 
 1   DEPARTAMENTO  363 non-null    object 
 2   AÑO           363 non-null    float64
 3   POBLACIÓN     363 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 14.2+ KB


In [17]:
tabla_wide = (
    data_dpto.pivot_table(
        index="DEPARTAMENTO",       # lo que queda como fila
        columns="AÑO",           # cada año será una columna
        values="POBLACIÓN",      # lo que se distribuye
        aggfunc="sum",           # suma (por si hay más de un registro)
        fill_value=0             # si falta valor, poner 0
    )
    .reset_index()
)

# Agregar columna con total por municipio
tabla_wide["TOTAL"] = tabla_wide.drop(columns="DEPARTAMENTO").sum(axis=1)

tabla_wide


AÑO,DEPARTAMENTO,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,TOTAL
0,AMAZONAS,69953,71215,72485,73790,75104,76573,78065,79583,80865,81996,83087,842716
1,ANTIOQUIA,6001151,6066806,6134953,6211312,6296843,6407149,6518070,6642072,6721140,6777249,6826125,70602870
2,ARAUCA,234529,236931,239772,244206,250502,262177,266031,270495,273137,274806,276240,2828826
3,"ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANT...",61543,61463,61406,61358,61309,61284,61679,62127,62573,62801,63007,680550
4,ATLÁNTICO,2341335,2366899,2393557,2427046,2468429,2535506,2608908,2693388,2741296,2774009,2805401,28155774
5,"BOGOTÁ, D.C.",7228427,7252949,7273265,7300918,7337449,7391056,7533202,7717564,7804920,7849206,7883928,82572884
6,BOLÍVAR,1961250,1977478,1993760,2013010,2035711,2070083,2108412,2151461,2177352,2195275,2210979,22894771
7,BOYACÁ,1181580,1187249,1193206,1199978,1207665,1218755,1233430,1248977,1260227,1268367,1275286,13474720
8,CALDAS,982563,982933,984360,987214,991363,998249,1011586,1025648,1035206,1041232,1044570,11084924
9,CAQUETÁ,395947,397383,398725,399983,401099,402085,406111,410567,414947,418724,422271,4467842


In [18]:
tabla_wide.to_csv(r"C:\Users\Hp\DENGUE\JBOOK\Poblacion\Poblacion_nacional.csv", index=False)


## Población departamental por municipio

In [9]:
data_mun = data_mun[(data_mun["AÑO"] >= 2017) & (data_mun["AÑO"] <= 2024)]

data_mun = data_mun[data_mun["ÁREA GEOGRÁFICA"] == "Total"]

data_mun['DEPARTAMENTO'] = data_mun['DEPARTAMENTO'].str.strip().str.upper()
data_mun['DEPARTAMENTO'] = data_mun['DEPARTAMENTO'].str.strip().str.upper()
data_mun['DEPARTAMENTO'] = data_mun['DEPARTAMENTO'].replace(
    {'ATLÃ¡NTICO': 'ATLÁNTICO', 'ATLÃƒÂ¡NTICO': 'ATLÁNTICO', 'ATLÃ\x83Â¡NTICO': 'ATLÁNTICO'})

data_mun = data_mun[data_mun['DEPARTAMENTO'] == 'ATLÁNTICO']
data_mun = data_mun.drop(columns=['ÁREA GEOGRÁFICA', 'DEPARTAMENTO']).reset_index(drop=True)

data_mun['MUNICIPIO'] = data_mun['MUNICIPIO'].str.strip().str.upper()
data_mun['MUNICIPIO'] = data_mun['MUNICIPIO'].replace(
    {'MANATÃ\xad':'MANATÍ', 'MANATÃÂ­': 'MANATÍ',
     'PIOJÃ³': 'PIOJÓ', 'PIOJÃÂ³': 'PIOJÓ',
     'REPELÃ³N': 'REPELÓN', 'REPELÃÂ³N': 'REPELÓN', 
     'SANTO TOMÃ¡S': 'SANTO TOMÁS', 'SANTO TOMÃÂ¡S': 'SANTO TOMÁS',
     'TUBARÃ¡':'TUBARÁ', 'TUBARÃÂ¡': 'TUBARÁ',
     'USIACURÃ\xad': 'USIACURÍ', 'USIACURÃÂ­': 'USIACURÍ',
     'SANTA LUCÃ\xadA': 'SANTA LUCÍA', 'SANTA LUCÃÂ­A': 'SANTA LUCÍA',
     })
data_mun['POBLACIÓN'] = (data_mun['POBLACIÓN'].astype(str).str.replace(',', '', regex=False).astype(float))


In [14]:
tabla_wide_mun = (
    data_mun.pivot_table(
        index="MUNICIPIO",       # lo que queda como fila
        columns="AÑO",           # cada año será una columna
        values="POBLACIÓN",      # lo que se distribuye
        aggfunc="sum",           # suma (por si hay más de un registro)
        fill_value=0             # si falta valor, poner 0
    )
    .reset_index()
)

# Agregar columna con total por municipio
tabla_wide_mun["TOTAL"] = tabla_wide_mun.drop(columns="MUNICIPIO").sum(axis=1)

tabla_wide_mun

AÑO,MUNICIPIO,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0,TOTAL
0,BARANOA,60676.0,62371.0,64550.0,66927.0,68800.0,70464.0,72072.0,73740.0,539600.0
1,BARRANQUILLA,1180860.0,1206322.0,1230835.0,1260910.0,1271797.0,1274895.0,1277216.0,1279120.0,9981955.0
2,CAMPO DE LA CRUZ,22604.0,22876.0,23641.0,24454.0,25124.0,25669.0,26217.0,26781.0,197366.0
3,CANDELARIA,15543.0,15977.0,16579.0,17240.0,17758.0,18197.0,18647.0,19104.0,139045.0
4,GALAPA,58493.0,60724.0,62945.0,65478.0,66754.0,67560.0,68313.0,69030.0,519297.0
5,JUAN DE ACOSTA,20366.0,21005.0,21596.0,22270.0,22748.0,23149.0,23545.0,23939.0,178618.0
6,LURUACO,27455.0,28173.0,29047.0,30001.0,30713.0,31315.0,31844.0,32443.0,240991.0
7,MALAMBO,124251.0,128199.0,133000.0,138416.0,141679.0,144155.0,146583.0,149019.0,1105302.0
8,MANATÍ,19426.0,19803.0,20521.0,21325.0,21908.0,22405.0,22882.0,23362.0,171632.0
9,PALMAR DE VARELA,28113.0,28928.0,30007.0,31160.0,32090.0,32941.0,33778.0,34647.0,251664.0


In [15]:
tabla_wide_mun.to_csv(r"C:\Users\Hp\DENGUE\JBOOK\Poblacion\Poblacion_departamental.csv", index=False)
